#Telecom Domain Read & Write Ops Assignment - Building Datalake & Lakehouse
This notebook contains assignments to practice Spark read options and Databricks volumes. <br>
Sections: Sample data creation, Catalog & Volume creation, Copying data into Volumes, Path glob/recursive reads, toDF() column renaming variants, inferSchema/header/separator experiments, and exercises.<br>

![](https://fplogoimages.withfloats.com/actual/68009c3a43430aff8a30419d.png)
![](https://theciotimes.com/wp-content/uploads/2021/03/TELECOM1.jpg)

##First Import all required libraries & Create spark session object

##1. Write SQL statements to create:
1. A catalog named telecom_catalog_assign
2. A schema landing_zone
3. A volume landing_vol
4. Using dbutils.fs.mkdirs, create folders:<br>
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/
5. Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):<br>
a. Volume vs DBFS/FileStore<br>
b. Why production teams prefer Volumes for regulated data<br>

##Data files to use in this usecase:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''

tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
'''

##2. Filesystem operations
1. Write dbutils.fs code to copy the above datasets into your created Volume folders:
Customer → /Volumes/.../customer/
Usage → /Volumes/.../usage/
Tower (region-based) → /Volumes/.../tower/region1/ and /Volumes/.../tower/region2/

2. Write a command to validate whether files were successfully copied

##3. Spark Directory Read Use Cases
1. Read all tower logs using:
Path glob filter (example: *.csv)
Multiple paths input
Recursive lookup

2. Demonstrate these 3 reads separately:
Using pathGlobFilter
Using list of paths in spark.read.csv([path1, path2])
Using .option("recursiveFileLookup","true")

3. Compare the outputs and understand when each should be used.

##4. Schema Inference, Header, and Separator
1. Try the Customer, Usage files with the option and options using read.csv and format function:<br>
header=false, inferSchema=false<br>
or<br>
header=true, inferSchema=true<br>
2. Write a note on What changed when we use header or inferSchema  with true/false?<br>
3. How schema inference handled “abc” in age?<br>

##5. Column Renaming Usecases
1. Apply column names using string using toDF function for customer data
2. Apply column names and datatype using the schema function for usage data
3. Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data 

## Spark Write Operations using 
- csv, json, orc, parquet, delta, saveAsTable, insertInto, xml with different write mode, header and sep options

##6. Write Operations (Data Conversion/Schema migration) – CSV Format Usecases
1. Write customer data into CSV format using overwrite mode
2. Write usage data into CSV format using append mode
3. Write tower data into CSV format with header enabled and custom separator (|)
4. Read the tower data in a dataframe and show only 5 rows.
5. Download the file into local from the catalog volume location and see the data of any of the above files opening in a notepad++.

1. Write customer data into CSV format using overwrite mode.

In [0]:
cust_csv=spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv.csv",header=True,inferSchema=True)
cust_csv.show()


In [0]:
cust_csv.write.csv("/Volumes/sparkio/etlschema/etlvolume/etl_dir/csvout/customer_csv",mode="overwrite")

2. Write usage data into CSV format using append mode   --------------pending with sep operation

In [0]:
usage_csv=spark.read.option("sep","\t").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_tsv.csv",header=True,inferSchema=True)
usage_csv.show()

In [0]:
usage_csv.write.csv("/Volumes/sparkio/etlschema/etlvolume/etl_dir/csvout/usage_csv",header="True",mode="append")

3. Write tower data into CSV format with header enabled and custom separator (|)

In [0]:
tower_csv=spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/tower_logs_region1.csv",header=True,sep='|',inferSchema=True)
tower_csv.show()

In [0]:
tower_csv.write.csv(path="/Volumes/sparkio/etlschema/etlvolume/etl_dir/csvout/tower_csv",header="True",sep='|',mode="overwrite")

In [0]:
tower_read_df1=spark.read.option("sep","|").csv(path="/Volumes/sparkio/etlschema/etlvolume/etl_dir/csvout/tower_csv/part-00000-tid-5273861698648449972-d2b3764c-81aa-48dc-bd68-f8d6ad664f04-230-1-c000.csv", header=True,samplingRatio=0.05)
display(tower_read_df1)

##7. Write Operations (Data Conversion/Schema migration)– JSON Format Usecases
1. Write customer data into JSON format using overwrite mode
2. Write usage data into JSON format using append mode and snappy compression format
3. Write tower data into JSON format using ignore mode and observe the behavior of this mode
4. Read the tower data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
cust_csv.write.json(path="/Volumes/sparkio/etlschema/etlvolume/etl_dir/jsonout/cust_json/",mode="overwrite")

In [0]:
usage_csv.write.json(path="/Volumes/sparkio/etlschema/etlvolume/etl_dir/jsonout/usage_json/",mode="append")

In [0]:
usage_csv.write.json(path="/Volumes/sparkio/etlschema/etlvolume/etl_dir/jsonout/usage_json/",mode="overwrite",compression="snappy")

In [0]:
usage_csv.write.json(path="/Volumes/sparkio/etlschema/etlvolume/etl_dir/jsonout/usage_json/",mode="append",compression="gzip")

In [0]:
tower_csv.write.json(path="/Volumes/sparkio/etlschema/etlvolume/etl_dir/jsonout/tower_json/",mode="ignore")

In [0]:
tower_csv.write.json(path="/Volumes/sparkio/etlschema/etlvolume/etl_dir/jsonout/tower_json/",mode="ignore",compression="gzip")

above ignore mode is just ignoring the file update if the directory or table/file already there 

In [0]:
tower_read_df1=spark.read.json(path="/Volumes/sparkio/etlschema/etlvolume/etl_dir/jsonout/tower_json/",samplingRatio=0.05)
display(tower_read_df1)

##8. Write Operations (Data Conversion/Schema migration) – Parquet Format Usecases
1. Write customer data into Parquet format using overwrite mode and in a gzip format
2. Write usage data into Parquet format using error mode
3. Write tower data into Parquet format with gzip compression option
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
cust_csv.write.parquet(path="/Volumes/sparkio/etlschema/etlvolume/etl_dir/parquetout/cust_parquet/",mode="overwrite",compression="gzip")

In [0]:
usage_csv.write.parquet(path="/Volumes/sparkio/etlschema/etlvolume/etl_dir/parquetout/usage_parquet/",mode="error")

In [0]:
usage_read_csv=spark.read.parquet("/Volumes/sparkio/etlschema/etlvolume/etl_dir/parquetout/usage_parquet/",header='True',inferSchema="True",samplingRatio=0.05)
display(usage_read_csv)

In [0]:
tower_csv.write.parquet(path="/Volumes/sparkio/etlschema/etlvolume/etl_dir/parquetout/tower_parquet/",compression="gzip")

##9. Write Operations (Data Conversion/Schema migration) – Orc Format Usecases
1. Write customer data into ORC format using overwrite mode
2. Write usage data into ORC format using append mode
3. Write tower data into ORC format and see the output file structure
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
cust_csv.write.orc(path="/Volumes/sparkio/etlschema/etlvolume/etl_dir/orcout/cust_orc/",mode="overwrite")

In [0]:
usage_csv.write.orc(path="/Volumes/sparkio/etlschema/etlvolume/etl_dir/orcout/usage_orc/",mode="append")

In [0]:
tower_csv.write.orc(path="/Volumes/sparkio/etlschema/etlvolume/etl_dir/orcout/tower_orc/",mode="overwrite")
tower_read_df1=spark.read.orc(path="/Volumes/sparkio/etlschema/etlvolume/etl_dir/orcout/tower_orc/")
display(tower_read_df1)
tower_read_df1.printSchema()

In [0]:
usage_read_orc=spark.read.option("samplingRation","0.05").orc("/Volumes/sparkio/etlschema/etlvolume/etl_dir/orcout/usage_orc/")
display(usage_read_orc)

##10. Write Operations (Data Conversion/Schema migration) – Delta Format Usecases
1. Write customer data into Delta format using overwrite mode
2. Write usage data into Delta format using append mode
3. Write tower data into Delta format and see the output file structure
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.
6. Compare the parquet location and delta location and try to understand what is the differentiating factor, as both are parquet files only.

In [0]:
cust_csv.write.format("delta").save(path="/Volumes/sparkio/etlschema/etlvolume/etl_dir/deltaout/cust_delta/",mode="overwrite")

In [0]:
usage_csv.write.format("delta").save(path="/Volumes/sparkio/etlschema/etlvolume/etl_dir/deltaout/usage_delta/",mode="append")

In [0]:
tower_csv.write.format("delta").save(path="/Volumes/sparkio/etlschema/etlvolume/etl_dir/deltaout/tower_delta/",mode="overwrite")
tower_read_df2=spark.read.format("delta").load("/Volumes/sparkio/etlschema/etlvolume/etl_dir/deltaout/tower_delta/")
display(tower_read_df2)
tower_read_df2.printSchema()

In [0]:
usage_read_delta=spark.read.format("delta").load("/Volumes/sparkio/etlschema/etlvolume/etl_dir/deltaout/usage_delta/",header="True",samplingRatio="0.05")
display(usage_read_delta)

##11. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data using saveAsTable() as a managed table
2. Write usage data using saveAsTable() with overwrite mode
3. Drop the managed table and verify data removal
4. Go and check the table overview and realize it is in delta format in the Catalog.
5. Use spark.read.sql to write some simple queries on the above tables created.


In [0]:
cust_csv.write.saveAsTable("sparkio.etlschema.cust_tab",mode='overwrite')

In [0]:
usage_csv.write.saveAsTable("sparkio.etlschema.usage_table",mode='overwrite')

In [0]:
%sql
drop table if exists sparkio.etlschema.cust_table

In [0]:
%sql
show tables in sparkio.etlschema

####Above Tables are created in managed and delta format tables.

In [0]:
df2=spark.sql("select * from sparkio.etlschema.cust_tab")
display(df2)
          

##12. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data using insertInto() in a new table and find the behavior
2. Write usage data using insertTable() with overwrite mode

In [0]:
cust_csv.write.insertInto("sparkio.etlschema.cust_tab",overwrite=True)

In [0]:
usage_csv.write.insertInto("sparkio.etlschema.usage_table",overwrite=True)

In [0]:
new_data = [(108,"selvam",36,"bangalore","prepaid"), (109,"sai","03","hyderabad","postpaid")]
newdf = spark.createDataFrame(new_data, ["customer_id", "name", "age", "city", "plan"])
newdf.write.mode("append").insertInto("sparkio.etlschema.cust_tab")
spark.sql("select * from sparkio.etlschema.cust_tab").display()

##13. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data into XML format using rowTag as cust
2. Write usage data into XML format using overwrite mode with the rowTag as usage
3. Download the xml data and open the file in notepad++ and see how the xml file looks like.

In [0]:
cust_csv.write.xml("/Volumes/sparkio/etlschema/etlvolume/etl_dir/xmlout/custxml/",rowTag="custum",mode="overwrite")

In [0]:
usage_csv.write.xml("/Volumes/sparkio/etlschema/etlvolume/etl_dir/xmlout/usagexml/",rowTag="usage",mode="overwrite")

##14. Compare all the downloaded files (csv, json, orc, parquet, delta and xml) 
1. Capture the size occupied between all of these file formats and list the formats below based on the order of size from small to big.

csvout:

customer file size=139.0 B  <br/>
usage=110 b<br/>
tower=144 b <br/>


Jsonout: <br/>

customer=369 b<br/>
usage=159 b <br/>
tower=232 b <br/>

Orcout: <br/>

Customer=925 b <br/>
usage=558 b <br/>
tower=795 b <br/>

Parquetout: <br/>

Customer:1.84 kb <br/>
usage:1.27 KB
tower=1.85 KB 

deltaout:

customer=1.50 KB
usage=1.27 KB
tower=1.47 KB

Xmlout:

customer=
usage=894

##15. Do a final exercise of defining one/two liner of... 
1. When to use/benifits csv
2. When to use/benifits json
3. When to use/benifit orc
4. When to use/benifit parquet
5. When to use/benifit delta
6. When to use/benifit xml
7. When to use/benifit delta tables
